In [ ]:
def abstract(file_path):
    # file_path = '/home/xyl/final/data/SRR11615792_1.fastq.gz'
    seq = []
    q = [] 
    with gzip.open(file_path,'rt') as handle:
        for record in SeqIO.parse(handle, 'fastq'):
            sequence = record.seq
            seq.append(sequence)
            quality = record.letter_annotations['phred_quality']
            q.append(quality)
        handle.close()
    return seq, q

In [ ]:
def select(seq, q, n):
    # 随机挑选n条
    nums = random.sample(range(0,len(seq)), n)
    new_seq = []
    new_q = []
    for i in range(n):
        new_seq.append(seq[nums[i]])
        new_q.append(q[nums[i]])
    return new_seq, new_q

In [ ]:
def q_subtract(seq, q):
    # 计算平均质量，选出不合格的，删去，返回删减完的seq
    # 计算平均质量
    ave_quality = [] # 存储各read平均质量
    for i in range(len(q)):
        a = q[i]
        sum = 0
        for j in range(len(a)):
            sum += a[j]
        ave_q = sum/(len(a))
        ave_quality.append(ave_q)

    # 选出不合格的
    number = [] # 存储不合格的编号
    for i in range(len(ave_quality)):
        a = ave_quality[i]
        if a<30:
            number.append(i)
    # print(len(number))

    # 在序列和质量列表中，删除不合格的read
    for i in range(len(number)):
        del seq[number[i]-i]
        # del q[number[i]-i]
    # print(len(seq))
    return seq

In [ ]:
def lenth75(seq):
    # 整理序列长度至75
    new_seq = [] # 存储最终的序列，n*75
    # new_q = []
    dif = []
    for i in range(len(seq)):
        a = seq[i] # 该条序列
        # b = q[i] # 该条质量值
        if len(a)<75: #短的不要
            continue
        elif 'N' in a: # 有模糊基的不要
            continue
        difference = len(a)-75 # 这一条多多少删多少
        dif.append(difference)
        if difference <=5:
            # 全删前面
            new_seq.append(a[difference::])
            # new_q.append(b[difference::])
        elif difference <=10:
            #前面删五个其他的在后面删
            new_seq.append(a[5:(5-difference)])
            # new_q.append(b[5:(5-difference)])
        else:
            # 后面删五个，其他的都在前面删
            new_seq.append(a[difference-5:-5])
            # new_q.append(b[difference-5:-5])
    print(len(new_seq))
    return new_seq

In [ ]:
def encode(seq):
    # one-hot编码
    lstmseq = np.zeros((1500, 4, 75), dtype='int')
    for i in range(1500):
        tmp = seq[i]
        for j in range(75):
            if tmp[j] == 'A':
                lstmseq[i, :, j] = [1, 0, 0, 0]
            elif tmp[j] == 'T':
                lstmseq[i, :, j] = [0, 1, 0, 0]
            elif tmp[j] == 'C':
                lstmseq[i, :, j] = [0, 0, 1, 0]
            elif tmp[j] == 'G':
                lstmseq[i, :, j] = [0, 0, 0, 1]
    # print(lstmseq)
    return lstmseq

In [ ]:
def save_data(seq,filename):
    # 保存最终数据
    # filename = './data/lstmseq.pkl'
    with open(filename,'wb') as f:
        pickle.dump(seq,f)

In [ ]:
def read_label():
    # 读取所有数据标签
    path = '/home/zw/Data/cfDNA_data/PRJNA574555.txt'
    with open(path, 'r') as file:
        lines = file.readlines()
        labels = []
        for line in lines:
            line=line.strip().split('\t')
            labels.append(line)

    label = []
    for i in range(len(labels)):
        a = [labels[i][0],labels[i][-1]]
        label.append(a)
    # print(label[1])  # ['SRR5146869', 'crc.ct-1']
    return label

In [ ]:
def save_label(label, name):
    for i in range(len(label)):
        if label[i][0] in name:
            y = label[i][1]
            break
    # print(y) # crc.ct-1
    if 'crc' in y:
        y_label = 0 # 患者标0
    else:
        y_label = 1 # 健康人标1
    return y_label

In [ ]:
import os
import gzip
from Bio import SeqIO
import random
import numpy as np
import pickle

new_path = '/home/zw/Data/cfDNA_data/process/CRC'
labels = read_label()
path = '/home/zw/Data/cfDNA_data/PRJNA574555'

for f_path in os.listdir(path):
    if '_1.fastq.gz' in f_path:
        file_name = f_path.replace("_1.fastq.gz", ".pkl")
        if file_name not in os.listdir(new_path): # prevent repetition
            fullpath = path + "/" + f_path
            # abstract seq and q
            seq, q = abstract(fullpath)
            # select 6000 reads randomly
            if len(seq) < 6000: # drop what can't be picked out
                print(file_name + ' is so short!')
                continue
            seq, q = select(seq, q, 6000)
            # delete by q
            seq = q_subtract(seq, q)
            # delete by seq, and unified seq
            seq = lenth75(seq)
            # abstract 1000 reads
            seq, _ = select(seq, q, 1500)
            # one-hot encoding
            seq = encode(seq)
            # select lables
            y = save_label(labels, f_path)
            # save together with seq: [array, int]
            data = [seq, y]
            # save data
            file_pkl = new_path+ "/" + file_name
            save_data(data,file_pkl)
            
            print(file_name + ' over!')